### 基于SVM算法的意图识别
在这个项目里， 你需要完成一个意图识别的任务，主要使用的模型是SVM算法。使用的数据集是`SMP2018中文人机对话技术评测`，是由科大讯飞股份有限公司提供数据。具体的数据可以从以下的链接下载： https://worksheets.codalab.org/worksheets/0x27203f932f8341b79841d50ce0fd684f/#， 下载之后把是数据集解压在当前的工程的根目录下。 在这个任务里，你即将要完成的几个任务是：

> 1. 读取文件、展示数据、并做并将label映射为数字index， 以及将index映射为类别。 
> 2. 使用分词工具对用户query进行切分。
> 3. 下载停用词表：https://github.com/goto456/stopwords/blob/master/%E4%B8%AD%E6%96%87%E5%81%9C%E7%94%A8%E8%AF%8D%E8%A1%A8.txt，   
    读取， 并对分词后结果进行过滤
> 4. 统计词频，并过滤低频词
> 5. 计算`tfidf`
> 6. 下载`tiny-word-embedding`：https://github.com/Embedding/Chinese-Word-Vectors, 分别使用`tfidf`, `word2vec` 建立`SVM`模型， 并对比二者结果。

你需要完成的部分为标记为`TODO`的部分。 

另外，提交作业时候的注意点：
> 1. 不要试图去创建另外一个.ipynb文件，所有的程序需要在`starter_code.ipynb`里面实现。很多的模块已经帮你写好，不要试图去修改已经定义好的函数以及名字。 当然，自己可以按需求来创建新的函数。但一定要按照给定的框架来写程序，不然判作业的时候会出现很多问题。 
> 2. 上传作业的时候把整个文件解压成.zip文件（不要.rar格式），请不要上传图片文件，其他的都需要上传包括`README.md`。
> 3. 确保程序能够正常运行，我们支持的环境是`Python 3`,  千万不要使用`Python 2`
> 4. 上传前一定要确保完整性，批改过一次的作业我们不会再重新批改，会作为最终的分数来对待。 
> 5. 作业可以讨论，但请自己完成。让我们一起遵守贪心学院的`honor code`。


In [2]:
import numpy as np
import pandas as pd
import jieba  # 感谢这个magic package，你不必要担心如何分词。如果想了解细节，可以参考此文件。
import requests
import os
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer    
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from sklearn.svm import SVC
from sklearn import metrics
from tqdm import tqdm
import pickle
import warnings

warnings.filterwarnings('ignore')

In [3]:
#下载数据，数据集为SMP2018
#训练集数据保存在同目录下的train.json文件中，测试集数据保存在同目录下的test.json文件中
#也可以使用！ wget https://worksheets.codalab.org/rest/bundles/0x0161fd2fb40d4dd48541c2643d04b0b8/contents/blob/ 方式下载
if not os.path.exists('train.json'):
    trainData = requests.get("https://worksheets.codalab.org/rest/bundles/0x0161fd2fb40d4dd48541c2643d04b0b8/contents/blob/")
    with open("train.json", "wb") as f:
         f.write(trainData.content)
            
if not os.path.exists('test.json'):
    testData = requests.get("https://worksheets.codalab.org/rest/bundles/0x1f96bc12222641209ad057e762910252/contents/blob/")
    with open("test.json", "wb") as f:
         f.write(testData.content)

In [4]:
#读取数据至DataFrame中
train_df = pd.read_json("train.json").transpose()
test_df = pd.read_json("test.json").transpose()

In [5]:
# 先来查看一下数据，确保没有任何错误！
print ("训练数据和测试数据:", train_df.shape, test_df.shape)
print ("标签的种类: ", train_df.label.unique()) # 查看标签的个数以及标签种类，预计10个类别。
train_df.head()

训练数据和测试数据: (2299, 2) (770, 2)
标签的种类:  ['weather' 'map' 'cookbook' 'health' 'chat' 'train' 'calc' 'translation'
 'music' 'tvchannel' 'poetry' 'telephone' 'stock' 'radio' 'contacts'
 'lottery' 'website' 'video' 'news' 'bus' 'app' 'flight' 'epg' 'message'
 'match' 'schedule' 'novel' 'riddle' 'email' 'datetime' 'cinemas']


,query,label
0,今天东莞天气如何,weather
1,从观音桥到重庆市图书馆怎么走,map
2,鸭蛋怎么腌？,cookbook
3,怎么治疗牛皮癣,health
4,唠什么,chat


In [8]:
labelName = train_df.label.unique() #全部label列表

# TODO 实现文本label 与index的映射 hint : zip dict
int_list = range(len(labelName))
n_dict = dict(zip(labelName, int_list))

In [9]:
# TODO 查看label 与index 的映射关系
print(n_dict)


{'weather': 0, 'map': 1, 'cookbook': 2, 'health': 3, 'chat': 4, 'train': 5, 'calc': 6, 'translation': 7, 'music': 8, 'tvchannel': 9, 'poetry': 10, 'telephone': 11, 'stock': 12, 'radio': 13, 'contacts': 14, 'lottery': 15, 'website': 16, 'video': 17, 'news': 18, 'bus': 19, 'app': 20, 'flight': 21, 'epg': 22, 'message': 23, 'match': 24, 'schedule': 25, 'novel': 26, 'riddle': 27, 'email': 28, 'datetime': 29, 'cinemas': 30}


> `TODO`: 统计一下每一个类别的文本各出现了几次？ 这有利于分析样本是否平衡还是不平衡。所谓的`不平衡样本`是有些类别的样本特别多，有些类别的样本特别少。这会引起模型训练的准确率。所以很重要一开始就要去看样本每个类别的个数。假如样本种类很多，即可以画出一个曲线出来。 

In [27]:
# TODO 统计并展示每一个类别出现的次数 hint groupby().count() / value_counts()
train_df['label'].value_counts()
#train_df.groupby('label').count()

chat           455
cookbook       269
video          182
epg            107
poetry         102
tvchannel       71
stock           71
train           70
map             68
weather         66
music           66
message         63
telephone       63
flight          62
translation     61
news            58
health          55
website         54
app             53
riddle          34
contacts        30
schedule        29
bus             24
radio           24
match           24
novel           24
calc            24
cinemas         24
lottery         24
email           24
datetime        18
Name: label, dtype: int64

In [29]:
# TODO 实现将index 数字与文本label的映射， 预测结果需要对比文本label
train_df['labelIndex'] = train_df.label.map(n_dict)
test_df['labelIndex'] = test_df.label.map(n_dict)

# 查看 index 与 label 的映射关系


In [33]:
# TODO 将dataframe 中文本label转换为数字。 hint:  map 
train_df["labelIndex"] = train_df.label.map(n_dict)# TODO
test_df["labelIndex"] = test_df.label.map(n_dict)# TODO
test_df.head()
#train_df.head()

,query,label,labelIndex
0,毛泽东的诗哦。,poetry,10
1,有房有车吗微笑,chat,4
2,2013年亚洲冠军联赛恒广州恒大比赛时间。,match,24
3,若相惜不弃下一句是什么？,poetry,10
4,苹果翻译成英语,translation,7


In [35]:
# TODO 对数据中的文本进行分词 hint:  jieba.cut

    # jieba.cut 返回一个generator, 需要进行转换 hint: list

def query_cut(query):
    return [word for word in jieba.cut(query)]
    
    # TODO

train_df["queryCut"] = train_df["query"].apply(query_cut)
test_df["queryCut"] = test_df["query"].apply(query_cut)

In [36]:
# 查看分词结果

train_df.head()

,query,label,labelIndex,queryCut
0,今天东莞天气如何,weather,0,"[今天, 东莞, 天气, 如何]"
1,从观音桥到重庆市图书馆怎么走,map,1,"[从, 观音桥, 到, 重庆市, 图书馆, 怎么, 走]"
2,鸭蛋怎么腌？,cookbook,2,"[鸭蛋, 怎么, 腌, ？]"
3,怎么治疗牛皮癣,health,3,"[怎么, 治疗, 牛皮癣]"
4,唠什么,chat,4,"[唠, 什么]"


In [38]:
# TODO 读取停用词
with open("中文停用词表.txt","r", encoding='utf-8') as f:
    stopWords = f.read().splitlines()

# 查看停用词
stopWords[0:59]

['$',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '?',
 '_',
 '“',
 '”',
 '、',
 '。',
 '《',
 '》',
 '一',
 '一些',
 '一何',
 '一切',
 '一则',
 '一方面',
 '一旦',
 '一来',
 '一样',
 '一般',
 '一转眼',
 '万一',
 '上',
 '上下',
 '下',
 '不',
 '不仅',
 '不但',
 '不光',
 '不单',
 '不只',
 '不外乎',
 '不如',
 '不妨',
 '不尽',
 '不尽然',
 '不得',
 '不怕',
 '不惟',
 '不成',
 '不拘',
 '不料',
 '不是',
 '不比',
 '不然',
 '不特',
 '不独',
 '不管',
 '不至于',
 '不若']

In [39]:
# TODO 使用停止词过滤上一步分词结果

def rm_stop_word(wordList):
    result = []
    for word in wordList:
        if word not in stopWords:
            result.append(word)
    return result
    # TODO

train_df["queryCutRMStopWord"] = train_df["queryCut"].apply(rm_stop_word)
test_df["queryCutRMStopWord"] = test_df["queryCut"].apply(rm_stop_word)

In [40]:
# 查看过滤停止词后的结果

train_df.head()

,query,label,labelIndex,queryCut,queryCutRMStopWord
0,今天东莞天气如何,weather,0,"[今天, 东莞, 天气, 如何]","[今天, 东莞, 天气]"
1,从观音桥到重庆市图书馆怎么走,map,1,"[从, 观音桥, 到, 重庆市, 图书馆, 怎么, 走]","[观音桥, 重庆市, 图书馆, 走]"
2,鸭蛋怎么腌？,cookbook,2,"[鸭蛋, 怎么, 腌, ？]","[鸭蛋, 腌]"
3,怎么治疗牛皮癣,health,3,"[怎么, 治疗, 牛皮癣]","[治疗, 牛皮癣]"
4,唠什么,chat,4,"[唠, 什么]",[唠]


In [43]:
#  TODO 计算词频 hint collections.Counter()
allWords = [word for query in train_df.queryCutRMStopWord for word in query] #所有词组成的列表
freWord = Counter(allWords)#统计词频，一个字典，键为词，值为词出现的次数



In [45]:
# TODO 过滤低频词
highFreWords = [word for word in freWord.keys() if freWord[word]>3] #词频超过3的词列表
def rm_low_fre_word(query):
    result = []
    for word in query:
        if word in highFreWords:
            result.append(word)
    return result
    # TODO
    
#去除低频词
train_df["queryFinal"] = train_df["queryCutRMStopWord"].apply(rm_low_fre_word)
test_df["queryFinal"] = test_df["queryCutRMStopWord"].apply(rm_low_fre_word)

2. 计算`TFIDF`。
这部分是将文本数据转化为计算机可以识别的类型。 tf-idf是一种统计方法，用以评估一字词对于一个文件集或一个语料库中的其中一份文件的重要程度。字词的重要性随着它在文件中出现的次数成正比增加，但同时会随着它在语料库中出现的频率成反比下降。
$$w_{i,j} = tf_{i, j} * log(\frac{N}{df_{i}})$$


主要使用sklearn 实现， 详细文档参考： https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

`tfidf`有几个关键参数  
> ngram_range: tuple(min_n, max_n) 要提取的n-gram的n-values的下限和上限范围，在min_n <= n <= max_n区间的n的全部值  
> stop_words：string {'english'}, list, or None(default)   
    如果为english，用于英语内建的停用词列表  
    如果为list，该列表被假定为包含停用词，列表中的所有词都将从令牌中删除  
    如果None，不使用停用词。max_df可以被设置为范围[0.7, 1.0)的值，基于内部预料词频来自动检测和过滤停用词  
> max_df： float in range [0.0, 1.0] or int, optional, 1.0 by default    
    当构建词汇表时，严格忽略高于给出阈值的文档频率的词条，语料指定的停用词。如果是浮点值，该参数代表文档的比例，整型绝对计数值，如果词汇表不为None，此参数被忽略。  
> min_df：float in range [0.0, 1.0] or int, optional, 1.0 by default    
    当构建词汇表时，严格忽略低于给出阈值的文档频率的词条，语料指定的停用词。如果是浮点值，该参数代表文档的比例，整型绝对计数值，如果词汇表不为None，此参数被忽略。  
> max_features： optional， None by default    
    如果不为None，构建一个词汇表，仅考虑max_features--按语料词频排序，如果词汇表不为None，这个参数被忽略  
> norm：'l1', 'l2', or None,optional  
    范数用于标准化词条向量。None为不归一化  
> smooth_idf：boolean，optional  
    通过加1到文档频率平滑idf权重，为防止除零，加入一个额外的文档

In [52]:
# TODO 将分词且过滤后的文本数据转化为tfidf 形式： 
trainText = [' '.join(query) for query in train_df["queryFinal"]]
testText = [' '.join(query) for query in test_df["queryFinal"]]
allText = trainText+testText
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf = tfidf.fit_transform(allText)

# sklearn tfidf vector fit_transform

In [54]:
# 切分数据集 hint sklearn train_test_split()
trainLen = len(train_df)
train_x_tfidf = tfidf.toarray()[0:trainLen]
test_x_tfidf = tfidf.toarray()[trainLen:]
train_y_tfidf = train_df["labelIndex"]
test_y_tfidf = test_df["labelIndex"]

In [55]:
# 切分后的数据信息
print("train_x_tfidf.shape =",train_x_tfidf.shape)
print("train_y_tfidf.shape =",train_y_tfidf.shape)
print("test_x_tfidf.shape =",test_x_tfidf.shape)
print("test_y_tfidf.shape =",test_y_tfidf.shape)

train_x_tfidf.shape = (2299, 238)
train_y_tfidf.shape = (2299,)
test_x_tfidf.shape = (770, 238)
test_y_tfidf.shape = (770,)


In [56]:
# 读取embedding
with open("tiny_word2vec.pickle","rb") as f:
    word2vec = pickle.load(f)

In [62]:
#词向量举例
word2vec["今天"].shape

(300,)

3. 加载`word embedding`。
词嵌入是自然语言处理中语言模型与表征学习技术的统称。概念上而言，它是指把一个维数为所有词的数量的高维空间嵌入到一个维数低得多的连续向量空间中，每个单词或词组被映射为实数域上的向量。

有很多pretrained embedding， 有很多加载方式， `gensim` 提供了很多有用的功能， 详细文档参考： https://radimrehurek.com/gensim/models/word2vec.html  
> 1.你需要做的是使用加载好的pretrained embedding， 将过滤后的分词转换为相应的词向量  
> 2.可能存在一些单词不在pretrained 的单词中， 这时候可以使用numpy 随机生成相同纬度的向量  
> 3.一个句子会有多个单词， 对于模型input会不一致， 一种简单的方法是对一个句子的所有词向量求平均值。 最后结果应该是（1， embedding_size)

In [63]:
#导入预训练好的词向量，词向量来源：https://github.com/Embedding/Chinese-Word-Vectors
with open("tiny_word2vec.pickle","rb") as f:
    word2vec = pickle.load(f)
# 词向量举例
word2vec["今天"]

# TODO 将过滤后的分词文本转换为相同维度的向量
vocabulary = word2vec.keys()
def sentence2vec(query):
    result = []
    for word in query:
        if word in word2vec:
            result.append(word2vec[word])
    result = np.array(result)
    if len(result)>0:
        result = result.sum(axis=0)/len(result)
    else:
        result = np.zeros(shape=(300,))
    return result
    # TODO
    

In [65]:
# 将转换为词向量的数据， 切分为训练集， 验证集
train_x_vec = np.vstack(train_df["queryCutRMStopWord"].apply(sentence2vec))
test_x_vec = np.vstack(test_df["queryCutRMStopWord"].apply(sentence2vec))
train_y_vec = train_df["labelIndex"]
test_y_vec = test_df["labelIndex"]

In [67]:
# 查看切分后的数据信息
print("train_x_vec.shape =",train_x_vec.shape)
print("train_y_vec.shape =",train_y_vec.shape)
print("test_x_vec.shape =",test_x_vec.shape)
print("test_y_vec.shape =",test_y_vec.shape)

train_x_vec.shape = (2299, 300)
train_y_vec.shape = (2299,)
test_x_vec.shape = (770, 300)
test_y_vec.shape = (770,)


4. `支持向量机`（英语：`support vector machine`，常简称为`SVM`）是在分类与回归分析中分析数据的监督式学习模型与相关的学习算法。SVM模型是将实例表示为空间中的点，这样映射就使得单独类别的实例被尽可能宽的明显的间隔分开。然后，将新的实例映射到同一空间，并基于它们落在间隔的哪一侧来预测所属类别。除了进行线性分类之外，SVM还可以使用所谓的核技巧有效地进行非线性分类，将其输入隐式映射到高维特征空间中。

我们在此主要使用sklearn来建立`SVM`模型（感谢sklearn), 更多相关文档参考：https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html    
`SVM` 关键参数:  
> C：C-SVC的惩罚参数C?默认值是1.0  
C越大，相当于惩罚松弛变量，希望松弛变量接近0，即对误分类的惩罚增大，趋向于对训练集全分对的情况，这样对训练集测试时准确率很高，但泛化能力弱。C值小，对误分类的惩罚减小，允许容错，将他们当成噪声点，泛化能力较强。
> kernel ：核函数，默认是rbf，可以是‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’   
  　　0 – 线性：u'v  
 　　 1 – 多项式：(gamma*u'*v + coef0)^degree  
  　　2 – RBF函数：exp(-gamma|u-v|^2)  
  　　3 –sigmoid：tanh(gamma*u'*v + coef0)  
> degree ：多项式poly函数的维度，默认是3，选择其他核函数时会被忽略。  
> gamma ： ‘rbf’,‘poly’ 和‘sigmoid’的核函数参数。默认是’auto’，则会选择1/n_features  
> coef0 ：核函数的常数项。对于‘poly’和 ‘sigmoid’有用。  
> probability ：是否采用概率估计？.默认为False  
> shrinking ：是否采用shrinking heuristic方法，默认为true  
> decision_function_shape ：‘ovo’, ‘ovr’ or None, default=None3 

接下来你需要做的是：
> 1. 分别使用tfidf, embedding建立线性SVM，以及非线性SVM
> 2. 对比模型结果，选择最优模型

In [72]:
# TODO 使用tfidf 特征建立线性SVM模型 hint: SVC()
from sklearn.svm import SVC
tfidfLinearSVM= SVC(C=1, kernel='linear', decision_function_shape='ovr') 
tfidfLinearSVM.fit(train_x_tfidf, train_y_tfidf)
print(tfidfLinearSVM.predict(train_x_tfidf))
# 输出模型结果， accuracy,  F1_score

print('train accuracy %s' % metrics.accuracy_score(train_y_tfidf, tfidfLinearSVM.predict(train_x_tfidf)))
print('train F1_score %s' % metrics.f1_score(train_y_tfidf, tfidfLinearSVM.predict(train_x_tfidf),average="macro"))
print('test accuracy %s' % metrics.accuracy_score(test_y_tfidf, tfidfLinearSVM.predict(test_x_tfidf)))
print('test F1_score %s' % metrics.f1_score(test_y_tfidf, tfidfLinearSVM.predict(test_x_tfidf),average="macro"))

[ 0  4  4 ... 22  4 12]
train accuracy 0.727707698999565
train F1_score 0.7708505943829783
test accuracy 0.6831168831168831
test F1_score 0.6954557599168526


In [73]:
# TODO 使用tfidf 特征建立`rbf` SVM 模型
tfidfKernelizedSVM = SVC(C=1, kernel='rbf', decision_function_shape='ovr')
tfidfKernelizedSVM.fit(train_x_tfidf, train_y_tfidf)

# 输出模型结果， accuracy,  F1_score

print('train accuracy %s' % metrics.accuracy_score(train_y_tfidf, tfidfKernelizedSVM.predict(train_x_tfidf)))
print('train F1_score %s' % metrics.f1_score(train_y_tfidf, tfidfKernelizedSVM.predict(train_x_tfidf),average="macro"))
print('test accuracy %s' % metrics.accuracy_score(test_y_tfidf, tfidfKernelizedSVM.predict(test_x_tfidf)))
print('test F1_score %s' % metrics.f1_score(test_y_tfidf, tfidfKernelizedSVM.predict(test_x_tfidf),average="macro"))

train accuracy 0.745541539799913
train F1_score 0.7984100288786787
test accuracy 0.6883116883116883
test F1_score 0.7020022181879286


In [75]:
# TODO 使用embeding 特征建立线性SVM模型
word2vecLinearSVM= SVC(C=1, kernel='linear', decision_function_shape='ovr') 
word2vecLinearSVM.fit(train_x_vec, train_y_vec)

# 输出模型结果， accuracy,  F1_score

print('train accuracy %s' % metrics.accuracy_score(train_y_vec, word2vecLinearSVM.predict(train_x_vec)))
print('train F1_score %s' % metrics.f1_score(train_y_vec, word2vecLinearSVM.predict(train_x_vec),average="macro"))
print('test accuracy %s' % metrics.accuracy_score(test_y_vec, word2vecLinearSVM.predict(test_x_vec)))
print('test F1_score %s' % metrics.f1_score(test_y_vec, word2vecLinearSVM.predict(test_x_vec),average="macro"))

train accuracy 0.9778164419312745
train F1_score 0.9818051685775911
test accuracy 0.8467532467532467
test F1_score 0.8565368406833987


In [77]:
# TODO 使用embedding  特征建立`rbf` SVM模型
word2vecKernelizedSVM= SVC(C=1, kernel='rbf', decision_function_shape='ovr') 
word2vecKernelizedSVM.fit(train_x_vec, train_y_vec)

# 输出模型结果， accuracy,  F1_score

print('train accuracy %s' % metrics.accuracy_score(train_y_vec, word2vecKernelizedSVM.predict(train_x_vec)))
print('train F1_score %s' % metrics.f1_score(train_y_vec, word2vecKernelizedSVM.predict(train_x_vec),average="macro"))
print('test accuracy %s' % metrics.accuracy_score(test_y_vec, word2vecKernelizedSVM.predict(test_x_vec)))
print('test F1_score %s' % metrics.f1_score(test_y_vec, word2vecKernelizedSVM.predict(test_x_vec),average="macro"))

train accuracy 0.9386689865158765
train F1_score 0.9353701170287393
test accuracy 0.8428571428571429
test F1_score 0.840815869158248
